# [LVV-T2730] - StarTracker Pointing and Tracking Test - Forward Azimuth Pattern

Collect data with the StarTracker following the azimuth pattern -270, -180, -90, 0, 90, 180, 270 deg.  
Nominal at four elevation angle 15, 45, 75, 86.5 deg.  
Minimum at the three angle: 15, 45, 86.5 deg.  

This test is forseen the first of four tests takes about one summer night (7 hours) in full version and a bit more than 5hours in the shortened version.  

Can we track for 10 min without moving the dome?

**Requirements:**
 - All the MT components should be enabled.
 - Need the `notebooks_vandv` module installed.

Please, see the [README] file for the requirements to run this notebook.  
The log messages printed in this notebook are stored in the EFD Script using the `-TTTTMMDD` format, where `TTTT` are the four last digits of the test case, `MM` is the 0-padded month, and `DD` is the 0-padded day of execution.

[README]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/README.md
[LVV-T2730]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2730

## Setup

In [1]:
test_case = "LVV-T2730"
test_exec = "LVV-EXXXX"

# az_grid = [180, 150, 120, 90, 60, 30, 0, -30, -60, -90, -120, -150]
# el_grid = [20, 45, 60, 75] 

az_grid = [225, 135, 45, -45]
el_grid = [25, 35, 45, 55, 65, 75]

# 101 - Wide Camera
# 102 - Narrow Camera
# 103 - Fast Camera (DIMM)
camera_sal_indexes = [101, 102, 103]
exposure_times = [5., 4., 6.]  # s
base_msg = f"{test_case} {test_exec}:"

track_time = 45. 
sleep_time = 1.5

### Prepare Notebook

Start with importing libraries:

In [2]:
%load_ext autoreload
%autoreload 2

import asyncio
import logging
import sys
import yaml

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time

from lsst.ts import salobj
from lsst.ts.idl.enums.MTDome import SubSystemId
from lsst.ts.idl.enums.Script import ScriptState
from lsst.ts.observatory.control import RotType
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control.generic_camera import GenericCamera

from lsst.sitcom import vandv

---
Print out the execution info for future reference.

In [3]:
exec_info = vandv.ExecutionInfo()
print(exec_info)


Executed by b1quint on 2023-03-16T04:06:46.962.
  Running in yagan02 at summit



---
Use the `maintel/setup_mtcs.py` script in LOVE or the [LVV-T2344] to test case and notebook to setup all the main telescope components.  
This includes simulators as well as real hardware when available (this will depend on when the test is conducted at TTS or on level 3 or on the telescope):  

- pointing  
- mount ( with the CCW)  
- rotator  
- ready M1M3: raise mirror, turn on FB, clear forces. Note that if used at level 3, we need to have M1M3 LUT use mount telemetry  
- ready M2: turn on FB, clear forces. Note that if used at level 3, we need to have M2 LUT use mount telemetry  
- Get cam hex Ready: check config; make sure LUT is on and has valid inputs; make sure hex is at LUT position  
- Get M2 hex (simulator) Ready: check config; make sure LUT is on and has valid inputs; make sure hex is at LUT position  
- Finally, get the MTAOS CSC ready  

[LVV-T2344]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2344

---
Setup the logger, the Domain, and the remote for the telescope control.

In [4]:
logging.basicConfig(format="%(asctime)s %(name)s: %(message)s", level=logging.DEBUG)
log = logging.getLogger(f"{test_case}")

In [5]:
os.environ["LSST_DDS_HISTORYSYNC"] = "200"
domain = salobj.Domain()
print(f"My user_host is {domain.user_host!r}\n")

My user_host is 'b1quint@nb-b1quint'



---
The `Script` CSC is used to record test checkpoints and allow to easy search in the EFD.

In [6]:
script = salobj.Controller("Script", index=199)
await script.start_task

script.log.info("Testing Log")

Instanciate the MTCS.

In [7]:
mtcs = MTCS(domain=domain, log=script.log)
vandv.logger.add_filter_to_mtcs()
await mtcs.start_task

[None, None, None, None, None, None, None, None, None, None]

--- 
Initialize the GenericCamera and ComCam.  
The ComCam is used to ensure that we can take exposures using two cameras at the same time.  
You might need to modify the cell below. 

In [8]:
camera_list = []

for index in camera_sal_indexes:

    gencam = GenericCamera(domain=domain, index=index, log=script.log)
    await gencam.start_task
    # await gencam.enable()

    camera_list.append(gencam)

In [11]:
camera_list[0].camera

Remote(name=GenericCamera, index=101)

Run the cell below to hide all the messages related to the DDS and have a cleaner notebook.

In [ ]:
vandv.logger.add_filter_to_mtcs()

Default wait time in the steps.

In [ ]:
STDWAIT = 20

### Check CSC Status

In [ ]:
await mtcs.set_state(
    salobj.State.DISABLED, 
    components=[
        "mtmount", 
        "mtrotator",
        "mtptg",
    ]
)

In [ ]:
await mtcs.set_state(
    salobj.State.ENABLED, 
    components=[
        "mtmount",
        # "mtrotator", # Enable this only if using the Rotator Hardware
        "mtptg",
    ]
)

## Increase MTMount Log Verbosity
await mtcs.rem.mtmount.cmd_setLogLevel.set_start(level=15)

In [ ]:
## Comment this line if you want the CCW to follow the Rotator
await mtcs.enable_ccw_following()

In [ ]:
## Home MTMount Main Axes
await mtcs.rem.mtmount.cmd_homeBothAxes.start(timeout=300)

In [ ]:
# Ignore (set check to False) the following components:
list_of_cscs = [
    "mtaos",
    "mtdome",
    "mtdometrajectory",
    "mthexapod_1",
    "mthexapod_2",
    "mtm1m3",
    "mtm2",
    "mtrotator",
]

for csc in list_of_cscs:
    script.log.info(f"Setting {csc} check to False")
    setattr(mtcs.check, csc, False)

In [ ]:
await mtcs.assert_liveliness()

## Enable this only if all the MT CSCs are enabled
# await mtcs.assert_all_enabled()

In [ ]:
for cam in camera_list:
    await cam.assert_liveliness()
    await cam.assert_all_enabled()

### Confirm take image each camera

In [ ]:
reason = "header_check"
await camera_list[0].take_object(exptime=1., reason=reason)
await camera_list[1].take_object(exptime=1., reason=reason)
await camera_list[2].take_object(exptime=1., reason=reason)

### Confirm Take Images in Sync

In [ ]:
await vandv.slew_and_track.take_images_in_sync_for_time(
    camera_list, 
    exposure_times, 
    reason="test_take_images_in_sync", 
    tracktime=60,
    sleep=sleep_time,
)

### Other preparation

- LVV-T2713 (1.0) Establish TMA - StarTracker Axis Angle Reference

## Helper functions

We moved all the helper functions in this notebook to [`lsst.sitcom.vandv.slew_and_track`].  
The interesting ones for this notebook are:

* `vandv.slew_and_track.generate_azel_sequence`
* `vandv.slew_and_track.take_images_in_sync_for_time`

[`lsst.sitcom.vandv.slew_and_track`]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/python/lsst/sitcom/vandv/slew_and_track.py

## Data Acquisition

### Loop without TMA tracking and without Dome Following

The following cell is isolated to allow continuing the grid from a point where you stopped. 

In [ ]:
azel_raw = [(az, el) for az, el in vandv.slew_and_track.generate_azel_sequence(az_grid, el_grid, el_limit=90.)]

## Uncomment this to run backward azimuth grid
# azel_raw = azel_raw[::-1]

for i, (az, el) in enumerate(azel_raw):
    print(i, az, el)

---
Uncomment and edit the lines below if you need a smaller grid for now.

In [ ]:
# az_grid_tiny = [130]
# el_grid_tiny = [40]
# azel_raw = [(az, el) for az, el in generate_azel_sequence(az_grid_tiny, el_grid_tiny)]

---
Uncomment the line bellow and replace i with the position index from the loop below

In [ ]:
recover_index = 0
azel = azel_raw[recover_index::]
print(azel)

---
This is the main loop. See comments in the code for details.

In [ ]:
script.log.info(f"{base_msg} Serpent Walk Backward - Continue")

# Initialize current_az to protect the Dome. 
# We only send move commands if we are going to a different az.
current_az = 0

for i, (az, el) in enumerate(azel):
    
    ## All of the following steps should be included in the point_azel command
    ## - Point the Dome
    ## - Wait the Dome to arrive    
    ## - Point the TMA
    ## - Wait the TMA to arrive
    script.log.info(f"{base_msg} Data acquisition #{i + recover_index} at az={az} and el={el} - Start")
    
    ## Disable dome following for now
    script.log.info(f"{base_msg} Dome following - Disable")
    # await mtcs.disable_dome_following()    
    mtcs.check.mtdometrajectory = False
    
    ## Start moving the Dome (only if we are going to a new Az)
    if az != current_az:
        script.log.info(f"{base_msg} Moving Dome to az={az} - START")
        await mtcs.rem.mtdome.cmd_exitFault.set_start()
        await asyncio.sleep(5)
        await mtcs.rem.mtdome.cmd_moveAz.set_start(position=az, velocity=0)
        dome_task = asyncio.create_task(wait_for_dome_in_position())
    else:
        script.log.info(f"{base_msg} Keep Dome at az={az}")
    
    ## Point Az/El
    script.log.info(f"{base_msg} Point to az={az}, el={el} - Start")
    radec = mtcs.radec_from_azel(az, el)
    await mtcs.slew_icrs(ra=radec.ra, dec=radec.dec, rot=0, rot_type=RotType.Physical)
    script.log.info(f"{base_msg} Point to az={az}, el={el} - Done")    
    
    ## Wait until dome in position
    if az != current_az:
        await dome_task
        script.log.info(f"{base_msg} Moving Dome to az={az} - DONE")

    ## Sleep and engage dome breaks
    script.log.info(f"{base_msg} Dome Brakes - Start")
    await asyncio.sleep(10)
    sub_system_ids = SubSystemId.AMCS
    await mtcs.rem.mtdome.cmd_stop.set_start(engageBrakes=True, subSystemIds=sub_system_ids)
    script.log.info(f"{base_msg} Dome Brakes - Done")
    
    ## Take images in sync - this will increase overhead on the camera with shorter exposure time.
    script.log.info(f"{base_msg} Take image(s) - Start")
    await vandv.slew_and_track.take_images_in_sync_for_time(camera_list, 
                                                            exposure_times,
                                                            reason=f"{base_msg[:-1]}_{i:03d}", 
                                                            tracktime=track_time,
                                                            sleep=sleep_time)
    script.log.info(f"{base_msg} Take image(s) - Done")

    script.log.info(f"{base_msg} Data acquisition #{i + recover_index} at az={az} and el={el} - Done")
    
    # Store current az to avoid sending dome to same place
    current_az = az
    
    # ExitFault just in case
    await mtcs.rem.mtdome.cmd_exitFault.set_start()
    
script.log.info(f"{base_msg} Serpent Walk Backward - Done")

## Slew and track a single target using Ra/Dec

Read Ra/Dec and convert to Az/El:

In [ ]:
coord = SkyCoord("05h23m27.4s -69°43'59.5", unit=(u.hourangle, u.deg), frame="icrs")
script.log.info("Slew and track single target - Sky coordinate:", coord)

azel = mtcs.azel_from_radec(coord.ra, coord.dec)
az, el = azel.az.deg, azel.alt.deg
script.log.info("Slew and track single target - Local coordinates:", az, el)

In [ ]:
script.log.info("Slew and track single target - START")

## Disable dome following for now
script.log.info(f"{base_msg} Dome following - Disable")
await mtcs.disable_dome_following()    
mtcs.check.mtdometrajectory = False

## Start moving the Dome
script.log.info(f"{base_msg} Moving Dome to az={az} - START")
await mtcs.rem.mtdome.cmd_exitFault.set_start()
await asyncio.sleep(5)
await mtcs.rem.mtdome.cmd_moveAz.set_start(position=az, velocity=0)
dome_task = asyncio.create_task(wait_for_dome_in_position())

## Point Az/El
script.log.info(f"{base_msg} Point to az={az}, el={el} - Start")
radec = mtcs.radec_from_azel(az, el)
await mtcs.slew_icrs(ra=radec.ra, dec=radec.dec, rot=0, rot_type=RotType.Physical)
script.log.info(f"{base_msg} Point to az={az}, el={el} - Done")

## Wait until dome in position
await dome_task
script.log.info(f"{base_msg} Moving Dome to az={az} - DONE")

## Sleep and engage dome breaks
script.log.info(f"{base_msg} Dome Brakes - Start")
await asyncio.sleep(10)
sub_system_ids = SubSystemId.AMCS
await mtcs.rem.mtdome.cmd_stop.set_start(engageBrakes=True, subSystemIds=sub_system_ids)
script.log.info(f"{base_msg} Dome Brakes - Done")

await vandv.slew_and_track.take_images_in_sync_for_time(camera_list, 
                                                        exposure_times,
                                                        reason=f"{base_msg[:-1]}_single", 
                                                        tracktime=track_time,
                                                        sleep=sleep_time)
script.log.info(f"{base_msg} Take image(s) - Done")    
script.log.info(f"{base_msg} Data acquisition #{i} at az={az} and el={el} - Done")
await mtcs.rem.mtdome.cmd_exitFault.set_start()

script.log.info("Slew and track single target - Done")

## Slew and Long Track using Alt/Az

In [ ]:
az = 270
el = 25
script.log.info(f"Slew and track single target - Local coordinates: {az}, {el}")

In [ ]:
## Move TMA
radec = mtcs.radec_from_azel(az, el)
await mtcs.slew_icrs(ra=radec.ra, dec=radec.dec, rot=0, rot_type=RotType.Physical)

In [ ]:
## Move the Dome to the start posiiton and a bit (1 deg) off to give more slew to telescope
await mtcs.rem.mtdome.cmd_moveAz.set_start(position=az-1, velocity=0.0)

In [ ]:
await vandv.slew_and_track.take_images_in_sync_for_time(camera_list, 
                                                        exposure_times,
                                                        reason=f"{base_msg[:-1]}_{i:03d}", 
                                                        tracktime=track_time,
                                                        sleep=sleep_time)

## Dome Commands

The following commands allow to control the Dome. However, make sure it is in ENABLED state. Use LOVE to change its state.

### Move to position

In [ ]:
await mtcs.rem.mtdome.cmd_moveAz.set_start(position=186, velocity=0)

### Stop the Dome

In [ ]:
sub_system_ids = SubSystemId.AMCS
await dome.cmd_stop.set_start(engageBrakes=True, subSystemIds=sub_system_ids)

### Recover from fault when stopping

In [ ]:
await mtcs.rem.mtdome.cmd_exitFault.set_start()